### Libraries

In [2]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
from psutil import virtual_memory    
import datetime
import numpy as np                 

'machine learning imports'
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
%pip install tensorflow==2.10.0

In [7]:
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.14.0


### GPU

In [10]:
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
ram_gb = virtual_memory().total / 1e9
print(f'{gpu.s} with {round(ram_gb,1)} GB available RAM.')
!nvcc --version

NVIDIA GeForce RTX 4060 Laptop GPU with 33.3 GB available RAM.
/bin/bash: line 1: nvcc: command not found


### Load Data

In [13]:
# Define the augmentation method and the data directory
AUGMENTATION = 'SMOTE 
data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mqtt.topic_0.0.1', 
                                                       'Unnamed: 0', 
                                                       'mqtt.topic_Temperature_and_Humidity', 
                                                       'mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])   

In [ ]:
def load_and_validate_data(data_dir, augmentation='None'):
    """
    Load and validate training and test data based on the augmentation option.

    Parameters:
    - data_dir: str, path to the directory containing data files.
    - augmentation: str, augmentation option ('None', 'SMOTE', 'SMOTE-NC', 'RealTabFormer', 'GReaT').

    Returns:
    - df_train: pd.DataFrame, training dataset.
    - df_test: pd.DataFrame, test dataset.
    """

    # Define file paths based on augmentation option
    file_paths = {
        'None': {'train': 'EdgeIIot_train_100k.csv', 'test' : 'EdgeIIot_test.csv'},
        'SMOTE': {'train': 'EdgeIIot_train_100k_SMOTE.csv', 'test' : 'EdgeIIot_test.csv'},
        'SMOTE-NC': {'train': 'EdgeIIot_train_100k_SMOTE_NC.csv', 'test' : 'EdgeIIot_test.csv'},
        'RealTabFormer': {'train': 'EdgeIIot_train_100k_RealTabFormer.csv', 'test' : 'EdgeIIot_test.csv'},
        'GReaT': {'train': 'EdgeIIot_train_100k_GReaT.csv', 'test' : 'EdgeIIot_test.csv'},
    }

    # Validate augmentation option
    if augmentation not in file_paths:
        raise ValueError("AUGMENTATION option not recognized. Please choose between 'None', 'SMOTE', 'SMOTE-NC', 'RealTabFormer', or 'GReaT'.")

    # Load training data
    df_train_path = os.path.join(data_dir, file_paths[augmentation]['train'])
    df_train = pd.read_csv(df_train_path, low_memory=False)

    # Load test data
    df_test_path = os.path.join(data_dir, file_paths[augmentation]['test'])
    df_test = pd.read_csv(df_test_path, low_memory=False)

    # Validate if test data has the same columns as training data
    if not set(df_train.columns) == set(df_test.columns):
        raise ValueError("Columns in the test data do not match columns in the training data.")

    # Print information about datasets
    print(f"Training data shape: {df_train.shape}, Test data shape: {df_test.shape}")

    return df_train, df_test

# Example usage:
data_dir = os.path.abspath('../data')
AUGMENTATION = 'None'
df_train, df_test = load_and_validate_data(data_dir, augmentation=AUGMENTATION)


### Data Preparation

In [14]:
# Drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [21]:
if AUGMENTATION == 'None':
    # Concatenate X_train and X_test to ensure consistent encoding for both
    concatenated_data = pd.concat([X_train, X_test], axis=0)

    # Extract categorical features
    categorical_features = concatenated_data.select_dtypes(include="object").columns

    # Get the unique values of all categorical columns
    for col in concatenated_data[categorical_features].columns:
        unique_values = concatenated_data[col].unique()
        print(f'{col}: \n{unique_values}\n')

    # One-hot encode categorical features for both X_train and X_test
    concatenated_data_encoded = pd.get_dummies(concatenated_data, 
                                               columns=categorical_features, 
                                               drop_first=True, 
                                               dtype='int8')

    # Split the data back into X_train and X_test
    X_train = concatenated_data_encoded.iloc[:len(X_train)]
    X_test = concatenated_data_encoded.iloc[len(X_train):]
    
    print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')
    
    def get_unique_values(df, columns):
        unique_values = df[columns].apply(lambda x: ''.join(x.astype(str)), axis=1).unique()
        unique_values.sort()
        return unique_values

    http_referer_columns = [col for col in X_train.columns if 'http.referer' in col]
    http_request_version_columns = [col for col in X_train.columns if 'http.request.version' in col]
    dns_qry_name_len_columns = [col for col in X_train.columns if 'dns.qry.name.len' in col]
    mqtt_conack_flags_columns = [col for col in X_train.columns if 'mqtt.conack.flags' in col]
    mqtt_protoname_columns = [col for col in X_train.columns if 'mqtt.protoname' in col]
    http_request_method_columns = [col for col in X_train.columns if 'http.request.method' in col]

    unique_values_http_referer = get_unique_values(X_train, http_referer_columns)
    unique_values_http_request_version = get_unique_values(X_train, http_request_version_columns)
    unique_values_dns_qry_name_len = get_unique_values(X_train, dns_qry_name_len_columns)
    unique_values_mqtt_conack_flags = get_unique_values(X_train, mqtt_conack_flags_columns)
    unique_values_mqtt_protoname = get_unique_values(X_train, mqtt_protoname_columns)

    # Apply the same function to X_test
    unique_values_http_referer_test = get_unique_values(X_test, http_referer_columns)
    unique_values_http_request_version_test = get_unique_values(X_test, http_request_version_columns)
    unique_values_dns_qry_name_len_test = get_unique_values(X_test, dns_qry_name_len_columns)
    unique_values_mqtt_conack_flags_test = get_unique_values(X_test, mqtt_conack_flags_columns)
    unique_values_mqtt_protoname_test = get_unique_values(X_test, mqtt_protoname_columns)
    
    # Join the unique values from both train and test data and print them
    unique_values_http_referer = np.unique(np.concatenate((unique_values_http_referer, unique_values_http_referer_test)))
    unique_values_http_request_version = np.unique(np.concatenate((unique_values_http_request_version, unique_values_http_request_version_test)))
    unique_values_dns_qry_name_len = np.unique(np.concatenate((unique_values_dns_qry_name_len, unique_values_dns_qry_name_len_test)))
    unique_values_mqtt_conack_flags = np.unique(np.concatenate((unique_values_mqtt_conack_flags, unique_values_mqtt_conack_flags_test)))
    unique_values_mqtt_protoname = np.unique(np.concatenate((unique_values_mqtt_protoname, unique_values_mqtt_protoname_test)))

    print(f'Unique values for http.referer: \n{unique_values_http_referer}\n')
    print(f'Unique values for http.request.version: \n{unique_values_http_request_version}\n')
    print(f'Unique values for dns.qry.name.len: \n{unique_values_dns_qry_name_len}\n')
    print(f'Unique values for mqtt.conack.flags: \n{unique_values_mqtt_conack_flags}\n')
    print(f'Unique values for mqtt.protoname: \n{unique_values_mqtt_protoname}\n')
    

    # check if X_train and X_test have categorical features
    print(f'X_train categorical features: {X_train.select_dtypes(include="object").columns}')
    print(f'X_test categorical features: {X_test.select_dtypes(include="object").columns}')

http.request.method: 
['0.0' '0' 'GET' 'POST' 'TRACE' 'OPTIONS' 'SEARCH' 'PROPFIND' 'PUT']

http.referer: 
['0.0' '0' '127.0.0.1'
 '() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }'
 'TESTING_PURPOSES_ONLY']

http.request.version: 
['0.0' '0' 'HTTP/1.1' 'HTTP/1.0' 'script>alert(1)/script><\\" HTTP/1.1'
 '/etc/passwd|?data=Download HTTP/1.1'
 '-al&ABSOLUTE_PATH_STUDIP=http://cirt.net/rfiinc.txt?? HTTP/1.1'
 '-al&_PHPLIB[libdir]=http://cirt.net/rfiinc.txt?? HTTP/1.1' '-a HTTP/1.1'
 'Src=javascript:alert(\'Vulnerable\')><Img Src=\\" HTTP/1.1'
 "name=a><input name=i value=XSS>&lt;script>alert('Vulnerable')</script> HTTP/1.1"
 'By Dr HTTP/1.1' '> HTTP/1.1']

dns.qry.name.len: 
['0.0' '1.0' '0' '2.debian.pool.ntp.org' '1.debian.pool.ntp.org'
 '3.debian.pool.ntp.org' '0.debian.pool.ntp.org' 'raspberrypi.local'
 'null-null.local' '_googlecast._tcp.local']

mqtt.conack.flags: 
['0.0' '0' '0x00000000' '1574358' '1461589' '1461383' '1574359' '1461074'
 '1461384' '1461591']

mq

In [16]:
# # Concatenate X_train and X_test
# X_comb = pd.concat([X_train[categorical_features], X_test[categorical_features]], axis=0)

# # Apply one-hot encoding (get_dummies)
# X_comb_enc = pd.get_dummies(X_comb, columns=categorical_features, drop_first=True,dtype='int8')

# # Split back into X_train and X_test
# X_train_enc, X_test_enc = train_test_split(
#     X_comb_enc, test_size=len(X_test), random_state=42)

# # Print the shape of X_train_enc and X_test_enc
# print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')

X_train_enc shape: (536515, 46), X_test_enc shape: (381934, 46)


In [220]:
# # Drop columns categorical_features from X_train and X_test 
# X_train = X_train.drop(categorical_features, axis=1)
# X_test = X_test.drop(categorical_features, axis=1)

# # Concatenate X_train and X_test with X_train_enc and X_test_enc and drop index column
# X_train = pd.concat([X_train.reset_index(drop=True), X_train_enc.reset_index(drop=True)], axis=1)
# X_test = pd.concat([X_test.reset_index(drop=True), X_test_enc.reset_index(drop=True)], axis=1)

# # Print the shape of X_train and X_test
# print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (536515, 92), X_test shape: (381934, 92)


#### Label Encoding

In [47]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


#### One-hot encoding of labels

In [48]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

#### Standardization of Data

In [49]:
# Instantiate the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train = scaler.fit_transform(X_train)

# Transform the test data
X_test = scaler.transform(X_test)

# Mean and standard deviation of X_train and X_test
print(f'X_train mean: {X_train.mean():.2f}, X_train std: {X_train.std():.2f}')
print(f'X_test  mean:  {X_test.mean():.2f}, X_test  std: {X_test.std():.2f}')

X_train mean: -0.00, X_train std: 0.94
X_test  mean:  -0.01, X_test  std: 1.19


### Model Training

In [50]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# ReduceLROnPlateau callback
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                               factor=0.3,
                                               mode="min",
                                               patience=10,
                                               verbose=1,
                                               min_lr=1e-8)

# Checkpoint callback                                                
checkpoint = ModelCheckpoint('../checkpoints/neural_net/best_model_multiclass.h5', 
                              monitor='loss', 
                              save_best_only=True)

In [51]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)
/home/tiagociic/Projectos/inovmineral/project/notebooks/checkpoints/neural_net

# Train the model
history = model.fit(X_train, 
                    y_train, 
                    epochs=100, 
                    batch_size=512, 
                    callbacks=[monitor, checkpoint])

Epoch 1/100
1048/1048 [==============================] - 7s 6ms/step - loss: 0.4710 - accuracy: 0.8043 - lr: 0.0010
Epoch 2/100
  27/1048 [..............................] - ETA: 6s - loss: 0.3536 - accuracy: 0.8335

/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1048/1048 [==============================] - 6s 6ms/step - loss: 0.3426 - accuracy: 0.8373 - lr: 0.0010
Epoch 3/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3254 - accuracy: 0.8437 - lr: 0.0010
Epoch 4/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3159 - accuracy: 0.8471 - lr: 0.0010
Epoch 5/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3092 - accuracy: 0.8494 - lr: 0.0010
Epoch 6/100
1048/1048 [==============================] - 7s 6ms/step - loss: 0.3054 - accuracy: 0.8499 - lr: 0.0010
Epoch 7/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3036 - accuracy: 0.8504 - lr: 0.0010
Epoch 8/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3017 - accuracy: 0.8509 - lr: 0.0010
Epoch 9/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3029 - accuracy: 0.8507 - lr: 0.0010
Epoch 10/100
1048/1048 [==============================] - 6s 6ms/step - loss: 0.3010

### Model Evaluation

In [55]:
# predict probabilities for test set and get the index of the highest probability
predictions = tf.argmax(model.predict(X_test), axis=1)

11936/11936 [==============================] - 11s 919us/step


In [58]:
# Calculate metrics 
accuracy = metrics.accuracy_score(tf.argmax(y_test, axis=1), predictions)
precision_m = metrics.precision_score(tf.argmax(y_test, axis=1), predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(tf.argmax(y_test, axis=1), predictions, average='macro')
f1_score_m = metrics.f1_score(tf.argmax(y_test, axis=1), predictions, average='macro')
precision_w = metrics.precision_score(tf.argmax(y_test, axis=1), predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(tf.argmax(y_test, axis=1), predictions, average='weighted')
f1_score_w = metrics.f1_score(tf.argmax(y_test, axis=1), predictions, average='weighted')

print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision (macro): {:.2f}".format(precision_m))
print("Recall (macro): {:.2f}".format(recall_m))
print("F1 (macro): {:.2f}".format(f1_score_m))
print("Precision (weighted): {:.2f}".format(precision_w))
print("Recall (weighted): {:.2f}".format(recall_w))
print("F1 (weighted): {:.2f}".format(f1_score_w))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")

Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.95
Precision (macro): 0.86
Recall (macro): 0.77
F1 (macro): 0.78
Precision (weighted): 0.96
Recall (weighted): 0.95
F1 (weighted): 0.94
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [59]:
# create dictionary for results
results = {
    "model": "Neural Net",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/neural_net.csv')

#### Confusion Matrix

In [60]:
conf_mat = metrics.confusion_matrix(tf.argmax(y_test, axis=1), predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4700,0,0,81,1,0,0,0,0,0,0,0,0,0,0
DDoS_HTTP,0,9171,0,0,0,0,0,0,0,0,0,0,0,5,452
DDoS_ICMP,0,0,13463,0,5,33,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,24,0,25,23,5,68,0,0,0,0,0,0,0,0,1
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,2,0,3,0,0,0,272766,0,0,0,0,0,4,1
Password,0,0,0,0,0,0,0,0,1768,0,0,7446,894,0,0
